# Introdução

A sumarização de textos é muito útil para enfatizar uma ideia principal de um trecho de conteúdo. Dada uma entrada, isto é, o texto original, o algoritmo deve ser capaz de retornar uma versão reduzida contendo apenas as principais frases do texto, porém mantendo a linha de raciocínio original. Outra técnica frequentemente empregada na área de Processamento de Linguagem Natural é o cálculo de similaridade de textos, que nos permite comparar, dados dois textos e uma métrica, o quão similar eles são. Neste projeto, utilizamos ambas as técnicas para criar uma recomendação sumarizada para um vídeo de um conjunto pré definido.

# Objetivo

O presente trabalho possui dois objetivos principais:
1. Gerar uma base de dados contendo informações sobre vídeos no Youtube, incluindo suas legendas no idioma inglês.
2. Partindo-se de uma base gerada, criar uma recomendação sumarizada de outro vídeo semelhante ao vídeo de entrada. O vídeo de entrada pode não estar presente na base, porém a recomendação sempre estará na base.

# Metodologia

Para a geração da base, as legendas analisadas foram manualmente enviadas pelos criadores dos vídeos, e se limitam exclusivamente a versões no idioma inglês, pois é a linguagem mais utilizada na web, e consequentemente mais vídeos com legendas para este idioma estão disponíveis. Para isso, o dataset foi criado utilizando a técnica abordada no artigo *["Creating an NLP data set from YouTube subtitles"](https://medium.com/@morga046/creating-an-nlp-data-set-from-youtube-subtitles-fb59c0955c2)*. Criamos uma token de autenticação na Google Cloud Platform para a API Youtube v3, responsável por retornar metadados sobre os vídeos públicos do site. Afim de evitar limites de requisição na API, delegamos o download das legendas para o programa [`youtube-dl`](https://youtube-dl.org/). Diferentemente do artigo, nosso dataset foi obtido a partir de diversas pesquisas aleatórias (opções como `criteria=random` na função `youtube_search`), para que várias categorias fossem incluídas. A biblioteca `webvtt` extraiu o texto das legendas, inicialmente no formato "Web Video Text Tracks", e por fim salvamos o conteúdo em uma planilha do Excel, `./bases_coletadas/ids.xslx`. 
A função `youtube_search` foi aplicada diversas vezes, para que no final obtivessemos uma base grande o suficiente com 16.066 linhas, uma para cada vídeo (e legenda) analisado. A funcão `clean_csv` então "limpa" o arquivo, removendo ids dos títulos dos vídeos das legendas e separando-as em um outro arquivo. Das linhas iniciais, apenas restaram 7457 itens, pois nem todos os vídeos possuíam legendas em inglês. Salvamos o dataset final em `./bases_coletadas/subtitles.csv`

Após a obtenção do dataset, iniciamos a implantação do projeto em Python. Nele, utilizamos as bibliotecas `pandas`, `regex`, `numpy` e `sklearn`. Dessa forma, iniciamos a etapa do pré-processamento. Durante essa etapa, importamos a base gerada e criamos cópias para que pudéssemos realizar o procedimento desejado tanto com um Stemmer quanto com um Lemmatizer, avaliando as vantagens e desvantagens de cada um. Além disso, também foi criada uma `base_limpa`, utilizada posteriormente na etapa de sumarização das legendas. Realizamos, então, a normalização do texto e a extração de stopwords do corpus em inglês, incluindo também algumas onomatopéias customizadas. Foram removidos padrões que começavam com `&`, como por exemplo `&nsbp;` (um "non-breaking space" que aparece frequentemente nos arquivos de legendas) e outros elementos como `/b` e `/i`. Outros passos durante o pré-processamento foram a remoção de maiúsculas, substituição de números indicativos por tokens, indicações de interlocutor/sons especiais, símbolos especiais, caracteres especiais e remoção de excesso de espaços além da tokenização e aplicação de Stemming e Lemmatization nas bases para avaliação de qual seria mais eficiente dentro do algoritmo.
Também ajustamos os títulos dos vídeos, removendo caracteres desnecessários, e retiramos textos com uma só palavra/letra.
O pré-processamento levou ~725 segundos (12 minutos e 5 segundos) para o Stemmer e ~312 segundos (5 minutos e 12 segundos) para o Lemmatizer. Em seguida, foi feito o carregamento de bases já pré-processadas para agilizar a execução do algoritmo, e remoção de linhas vazias após pré-processamento.

Na última etapa, ocorreu a geração das matrizes tfidf com os termos para cada uma das bases pré-processadas. Após a realização de testes, observamos que seria mais vantajoso utilizar o Stemming durante o pré-processamento, utilizando, portanto, a base pré-processada através dessa técnica. O número de itens restantes após a filtragem para ambos foi de 7.306 legendas. Neste momento, definimos funções gerar a sumarização de texto, com a funções responsáveis por quebrar o texto em frases, calcular similaridade entre sentenças (utilizando a métrica de distância de cosseno), gerar uma matriz de similaridade e, por fim, "rankear" as sentenças e uní-las em um resumo. Por fim, definimos a função `find_most_similar_any_video`, que compara a legenda de um vídeo de entrada (cuja legenda será obtida utilizando o `youtube-dl` novamente) com as legendas de todos os vídeos da base e retorna um resumo do vídeo com maior similaridade de cosseno. O resumo é criado a partir da técnica de sumarização, abrangendo muitas funções utilizadas em aula. Por fim, a fim de evitar problemas quanto ao download de programas externos etc., também definimos a função `find_most_similar_video`, que utiliza a mesma métrica para calcular uma matriz de similaridade entre os textos da base e, portanto, faz a recomendação apenas para textos de dentro da base.

# Resultados Obtidos

Os resultados obtidos foram satisfatórios, porém apresentaram algumas limitações. Observamos que o algoritmo apresenta sugestões bem semelhantes para um bom número de vídeos. No entanto, vídeos relacionados a genética, por exemplo, apresentaram sugestões de vídeos de esportes e gincanas e apontaram alta similaridade. Algumas possibilidades para explicar esse fenômeno residem no fato das legendas representarem a fala das pessoas, cuja interpretação pode ser altamente dependente do contexto e de outros sinais não verbais. Com relação à sumarização de texto, o resultado também foi satisfatório. Alguns desafios que encontramos foi a ausência de pontuações em alguns vídeos, o que prejudicou a geração dos resumos. A fim de contornar o problema, utilizamos símbolos especiais, como `&nbsp;` (non-breaking space), como pontuação, e assim conseguimos resultados melhores.
Utiliando um exemplo com uma aula sobre política, uma aula de Big Data foi retornada, o que indica que a similaridade foi encontrada, pois o modelo percebeu a relação entre aulas, apesar de não encontrá-la entre tópicos. Um motivo pode ser a ausência de um vídeo mais específico sobre política em nossa base. Além disso, outro motivo pode ser o contexto específico com relação ao modo da fala do locutor, como ja comentado.
Utilizando outro exemplo com um vídeo de malhação a resposta também foi outro vídeo de continuação do mesmo autor, com alta similaridade (92%).

In [7]:
#!pip install os
import os
#!pip install pandas
import pandas as pd
#!pip install numpy
import numpy as np
#!pip install nltk
import nltk
#!pip install re
import re
#!pip install scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer


#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')
stopwords = nltk.corpus.stopwords.words('english') + ['oh', 'uh', 'na', 'gon', 'um', 'yeah', 'ih', 'meh', 'ai', 'hum', 'hehe', 'haha', 'hihi', 'hoho', 'hohoho'] #adicionando algumas expressões comuns que podem trazer problemas
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [8]:
path = os.getcwd() + '\\'
path_bases = path + 'bases_coletadas\\'

base_legendas = pd.read_csv(path_bases + 'subtitles.csv').dropna(axis=0).drop('Unnamed: 0', axis=1).drop_duplicates(subset=['video_captions'], keep='first').reset_index(drop=True)
base_legendas2 = base_legendas.copy(deep=True)
base_limpa = base_legendas.copy(deep=True)

In [9]:
#Definição de funções para pré-processamento

from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def stemming(lista):
    retorno = []
    for word in lista:
        retorno.append(stemmer.stem(word))
    return retorno

def lemming(lista):
    retorno = []
    for word in lista:
        retorno.append(lemmatizer.lemmatize(word))
    return retorno

def remover_simbolos(texto):
    texto = re.sub(r'&.*?;\w??&.*?;', ' ', texto) #retira caracteres presos entre &...; 
    texto = re.sub(r'&.*?;', ' ', texto) #retira &rslp; e afins
    texto = re.sub(r'\/\w', ' ', texto) #retira /b, /i etc.
    return texto
    
def preprocessamento(texto, reduc='stemming'):
    #Removendo maiúsculas...
    texto = texto.lower()
    #Substituindo numeros por token indicativo...
    texto = re.sub(r'[0-9]+', '_NUMBER_', texto)
    #Removendo indicações de interlocutor/sons especiais...
    texto = re.sub(r'[\[\(].*?[\]\)]', ' ', texto)
    #Removendo símbolos especiais:
    texto = remover_simbolos(texto)
    #Removendo caracteres especiais...
    texto = re.sub(r'[^a-z\s]', ' ', texto)
    texto = ' '.join(texto.split()) #retira excesso de espaços
    #Tokenização...
    tokens = nltk.tokenize.word_tokenize(texto)
    #Removendo letras soltas...
    tokens = [token for token in tokens if len(token) > 1]
    #Removendo stopwords...
    tokens_sw = [token for token in tokens if token not in stopwords]
        #Aplicando Stemming
    if reduc=='stemming':
        tokens_sw = stemming(tokens_sw)
    else:
        #Aplicando Lemming
        tokens_sw = lemming(tokens_sw)
    return ' '.join(tokens_sw)

def preprocessarbase(base, coluna, reduc='stemming'): #Executa o pré-processamento a todos os textos de uma base de dados
    
    base[coluna] = base[coluna].apply(lambda x: preprocessamento(x, reduc))
    
    return None

def ajustar_titulo(base, coluna): #Retira o ID e caracteres desnecessários dos títulos dos vídeos
    base_clone = base.copy(deep=True)
    titulos = list(base[coluna])
    for i in range(len(titulos)):
        titulo = titulos[i]
        titulo = titulo.split('-')
        titulo.pop(-1)
        titulo = ' '.join(titulo)
        titulo = re.sub(r'[^a-zA-Z#0-9_\s]', '', titulo)
        titulos[i] = titulo
    base[coluna] = titulos
    
    return None

def retirar_unicos(base, coluna): #retira textos com uma só palavra/letra
        index = list(base.index)
        remover = []
        for i in index:
            texto = base.loc[i, coluna]
            if len(texto) <= 1 or len(texto.split()) <=1:
                remover += [i]
        base.drop(remover, axis=0, inplace=True)

In [10]:
#Verificando a base
base_legendas.head()
base_legendas2.head()

,video_title,video_captions,video_id
0,#007_NewCityProductionsPodcast-CurtisMcCoshhow...,[Becky] Hi everybody welcome back to another e...,RX--L0xnnDo
1,#1-JoshGroban—TheLinkBetweenMusicandMentalHeal...,"(upbeat music) - [Marjorie] Hi, this\nis Marjo...",kNxmPdHVtp4
2,#16ទៅប្រមូលក្រីស្តាល😁_ARK-SurvivalEvolved-QLd1...,Hello Guys\nHope you guys enjoy this video\nDo...,QLd1nGvjks4
3,#193-KetoandHeartHealthwithDr.BretScher-ZIoXUp...,"Hey everyone, this is Geoffrey\nWoo with the h...",ZIoXUpceBb4
4,#19HowtoorderfoodinSpanish(Dialogue)-_lv-ooUR4...,Yourspanishguide.com Episode number 29. Hello ...,_lv-ooUR4lw


In [5]:
#Aplicando pré-processamento na base e ajuste nos títulos com stemming
from time import time
print("Inicio do pré-processamento dos dados...")
inicio = time()
preprocessarbase(base_legendas, 'video_captions')
ajustar_titulo(base_legendas, 'video_title')
retirar_unicos(base_legendas, 'video_captions')
fim = time()
print("Pré-processamento finalizado. Tempo decorrido: " + str(fim-inicio) + " segundos.")
base_legendas.to_csv(path_bases + 'Base_PREPROC_STEMMING.csv', index=False)

base_legendas.head()

Inicio do pré-processamento dos dados...
Pré-processamento finalizado. Tempo decorrido: 724.9425637722015 segundos.


,video_title,video_captions,video_id
0,#007_NewCityProductionsPodcast CurtisMcCoshhow...,hi everybodi welcom back anoth episod new citi...,RX--L0xnnDo
1,#1 JoshGrobanTheLinkBetweenMusicandMentalHealth,hi marjori morrison patrick kennedi welcom psy...,kNxmPdHVtp4
2,#16_ARK SurvivalEvolved,hello guy hope guy enjoy video forget leav like,QLd1nGvjks4
3,#193 KetoandHeartHealthwithDrBretScher,hey everyon geoffrey woo health via modern nut...,ZIoXUpceBb4
4,#19HowtoorderfoodinSpanishDialogue _lv,yourspanishguid com episod number hello hello ...,_lv-ooUR4lw


In [6]:
#Aplicando pré-processamento na base e ajuste nos títulos com lemming
print("Inicio do pré-processamento dos dados...")
inicio = time()
preprocessarbase(base_legendas2, 'video_captions', reduc='lemmatizing')
ajustar_titulo(base_legendas2, 'video_title')
retirar_unicos(base_legendas2, 'video_captions')
fim = time()
print("Pré-processamento finalizado. Tempo decorrido: " + str(fim-inicio) + " segundos.")
base_legendas2.to_csv(path_bases + 'Base_PREPROC_LEMMING.csv', index=False)
base_legendas2.head()

Inicio do pré-processamento dos dados...
Pré-processamento finalizado. Tempo decorrido: 312.62347626686096 segundos.


,video_title,video_captions,video_id
0,#007_NewCityProductionsPodcast CurtisMcCoshhow...,hi everybody welcome back another episode new ...,RX--L0xnnDo
1,#1 JoshGrobanTheLinkBetweenMusicandMentalHealth,hi marjorie morrison patrick kennedy welcome p...,kNxmPdHVtp4
2,#16_ARK SurvivalEvolved,hello guy hope guy enjoy video forget leave like,QLd1nGvjks4
3,#193 KetoandHeartHealthwithDrBretScher,hey everyone geoffrey woo health via modern nu...,ZIoXUpceBb4
4,#19HowtoorderfoodinSpanishDialogue _lv,yourspanishguide com episode number hello hell...,_lv-ooUR4lw


In [11]:
#Carregando bases já pré-processadas (para agilizar) e removendo vazios após pré-processamento
#base_legendas = pd.read_csv(path_bases + 'Base_PREPROC_STEMMING.csv')
#base_legendas2 = pd.read_csv(path_bases + 'Base_PREPROC_LEMMING.csv')

base_legendas.dropna(inplace=True, axis=0)
#base_legendas2.dropna(inplace=True, axis=0)

base_legendas.shape

(7293, 3)

In [12]:
#Gerando matrizes tf-idf com os termos para cada uma das bases pré-processadas
#lemming_tfidf = TfidfVectorizer(stop_words='english')
stemming_tfidf = TfidfVectorizer(stop_words='english')

XS_tfidf = stemming_tfidf.fit_transform(base_legendas['video_captions'])
#XL_tfidf = lemming_tfidf.fit_transform(base_legendas2['video_captions'])

print(XS_tfidf.shape)
#A partir daqui, dados os testes que efetuamos, utilizaremos apenas a base com pré-processamento via stemming.

(7293, 161669)


In [13]:
#Definindo funções para sumarização de texto

from nltk.cluster.util import cosine_distance
from nltk.tokenize import sent_tokenize
from nltk.tokenize.regexp import regexp_tokenize
#!pip install networkx
import networkx as nx

def read_text(text, n_sent=-1): #quebra texto em sentenças
    retorno = []
    sentences = sent_tokenize(text)
    if len(sentences) == 1:
        sentences = regexp_tokenize(text, r'&.*?;', gaps=True)
    for sentence in sentences:
        sentence = remover_simbolos(sentence)
        retorno += [sentence.replace('[^a-zA-Z]', ' ').split()]
    if len(retorno) < n_sent:
        return retorno
    
    return retorno[:n_sent]
    

def sentence_similarity(sent1, sent2, stopwords=None): #calcula similaridade entre duas sentenças
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    for w in sent1:
        if w not in stopwords:
            vector1[all_words.index(w)] += 1
 
    for w in sent2:
        if w not in stopwords:
            vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)    

def build_similarity_matrix(sentences, stop_words): #cria matriz de similaridade com base nas sentenças

    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 != idx2:
                similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    
    return similarity_matrix


def generate_summary(file_name, top_n=5, n_sent=-1): #gera resumo com base nas sentenças mais similares
    stop_words = stopwords
    summarize_text = []    
    
    #Aplica tokenização do texto em sentenças
    sentences =  read_text(file_name, n_sent=n_sent) 
    
    #Constrói a matrix de similaridade
    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)    

    #Cria "rank" de sentenças
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)  
    
    #Pega as sentenças com maior rank
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    
    if len(ranked_sentence) < top_n: #Para evitar erros em casos em que não é possível tokenizar o texto
        top_n=1

    for i in range(top_n):
        summarize_text.append(' '.join(ranked_sentence[i][1]))

    return ' '.join(summarize_text)

In [14]:
#Definindo função para recomendação de vídeos similares a partir de uma URL do YouTube.
from sklearn.metrics.pairwise import cosine_similarity
#!pip install youtube-dl
#!pip install webvtt-py
import webvtt

def get_captions(url): #Função que baixa a legenda de um vídeo de acordo com sua URL. Precisa da instalação do Youtube-DL
    lang='en'
    cmd = ["youtube-dl","--skip-download","--write-sub",
               "--sub-lang",lang,url]
    os.system(" ".join(cmd))


def convert_vtt(): #Converter a legenda de .vtt para .csv 
    try:
        file = [filename for filename in os.listdir() if filename[-3:] == 'vtt'][0]
    except:
        print('This video has no subtitles =(')
        return '_BREAK_'

    #create an assets folder if one does not yet exist
    if os.path.isdir('{}/assets'.format(os.getcwd())) == False:
        os.makedirs('assets')
    #extract the text and times from the vtt file
    captions = webvtt.read(file)
    text_time = pd.DataFrame()
    text_time['text'] = [caption.text for caption in captions]
    text_time['start'] = [caption.start for caption in captions]
    text_time['stop'] = [caption.end for caption in captions]
    text_time.to_csv('assets/{}.csv'.format(file[:-4]),index=False) #-4 to remove '.vtt'
    #remove files from local drive
    os.remove(file)
    return file[:-4]

def find_most_similar_any_video(url): #Compara a legenda do vídeo com a legenda de todos os vídeos da base e retorna um resumo do vídeo com maior similaridade.
    similarity = []
    get_captions(url)
    file = convert_vtt()
    if file == '_BREAK_':
        return None
        
    video = pd.read_csv(path + 'assets\\' + file + '.csv')
    video['text'] = video['text'].apply(lambda x: str(x))
    text = stemming_tfidf.transform([preprocessamento(' '.join(video['text']))])
    
    for i in range(XS_tfidf.shape[0]):
        row = XS_tfidf[i]
        simi = cosine_similarity(text, row)[0][0]
        if simi >= 0.99: #retira o próprio vídeo da contagem
            similarity += [0.0]
        else:
            similarity += [simi]
        
    index = similarity.index(max(similarity))
    
    
    id_video = base_limpa.loc[index, 'video_id']
    url_video = 'https://www.youtube.com/watch?v=' + id_video
    text_video = base_limpa.loc[index, 'video_captions']
    print()
    title_video = base_limpa.loc[index, 'video_title'].split('-')
    title_video.pop()
    title_video = ' '.join(title_video)
    resumo=generate_summary(text_video, n_sent=500)
    return title_video + '\n\n' + resumo  + '\n\n' + 'Available on: ' + url_video + '\n\n' + 'Similarity: ' + str(max(similarity))

In [18]:
print(find_most_similar_any_video('https://www.youtube.com/watch?v=GOq8-FR8s1E'))


CoronaVirus(COVID 19)discussionwithBillGates

- Well, there's the league of, once you do the shutdown, you need to go, at least maybe two infection periods before you'd really expect to see things going down. And I'll add that question, maybe make it global, how do people balance, I've heard the argument that the economic harm could could cause a lot of deaths too and you could imagine in places like India, I've been monitoring some of the news there of folks who, with the shutdown, they have no livelihood, and they don't know how they're gonna get food, so that's happening even in the US, so how do you weigh those tensions, and what do you see as the dynamics that's keeping us from a really serious shutdown versus some of the more scattershot things that have been put in place? - Yeah, now we're gonna talk more about that. - Thank you so much, We could obviously talk for hours, I'm sure many people would love to hear there's, we could tell there's just so much more that we could lear

In [16]:
similarity = cosine_similarity(XS_tfidf)
def find_most_similar_video(url): #Para comparações apenas de vídeos dentro da base de forma mais rápida
    values = []
    video_id = re.findall(r'v=(.+)', url)[0]
    video_index = list(base_legendas['video_id']).index(video_id)
    
    for index in range(XS_tfidf.shape[0]):
        simi = similarity[video_index][index]
        if simi >= 0.99:
            values += [0.0]
        else:
            values += [simi]
    index = values.index(max(values))
    
    id_video = base_limpa.loc[index, 'video_id']
    url_video = 'https://www.youtube.com/watch?v=' + id_video
    text_video = base_limpa.loc[index, 'video_captions']
    title_video = base_limpa.loc[index, 'video_title'].split('-')
    title_video.pop()
    title_video = ' '.join(title_video)
    resumo = generate_summary(text_video, n_sent=500)
    return title_video + '\n\n' + resumo + '\n\n' + 'Available on: '+ url_video + '\n\n' + 'Similarity: ' + str(max(values))

In [17]:
print(find_most_similar_video('https://www.youtube.com/watch?v=PmMISnFqg8E'))

FullChairWorkout NoEquipment,Seated_MoreLifeHealth

those shoulders up back and down and we're going to do that five times let's go one two three four and five - excellent work let's bring those arms up like this just to 90 degrees nice gentle posture what we're going to do are some shoulder circles backwards let's go one two and three - excellent work now we're just breathe keeping those arms pumping five seconds you can do it putting it in and three two and one - excellent work taking a deep breath in and out okay now we're going to go back to circles and let's go for one two three four and five excellent work bring the arms down like front of the arms five six seven eight nine - now let's go down slow now let's go one more coming up and down - excellent work let's shake out those arms taking a deep breath in and out now sitting with those shoulders back and down keeping that stomach tight let's face our hands

Available on: https://www.youtube.com/watch?v=hzYCL86BFH8

Similarity: 0.